In [1]:
import pandas as pd
import pickle
import torch
import os
import re
import random
import csv
import json
import jsonlines
import numpy as np
import pickle
import time
import gzip
from tqdm import tqdm, trange
from sklearn.cluster import KMeans
from typing import Any, List, Sequence, Callable
from itertools import islice, zip_longest
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.cluster import MiniBatchKMeans
from collections import defaultdict

## Origina data transformation

###### Download NQ Train and Dev dataset from https://ai.google.com/research/NaturalQuestions/download
###### NQ Train: https://storage.cloud.google.com/natural_questions/v1.0-simplified/simplified-nq-train.jsonl.gz
###### NQ Dev: https://storage.cloud.google.com/natural_questions/v1.0-simplified/nq-dev-all.jsonl.gz

In [2]:
nq_dev = []

with gzip.open("/home/ec2-user/quic-efs/user/bowenjin/seq2seq/retrieval-data/NQ/v1.0-simplified_nq-dev-all.jsonl.gz", "r+") as f:
    for item in tqdm(jsonlines.Reader(f)):
        
        arr = []
        ## question_text
        question_text = item['question_text']
        arr.append(question_text)

        tokens = []
        for i in item['document_tokens']:
            tokens.append(i['token'])
        document_text = ' '.join(tokens)
        
        ## example_id
        example_id = str(item['example_id'])
        arr.append(example_id)

        # document_text = item['document_text']
        ## long_answer
        annotation = item['annotations'][0]
        has_long_answer = annotation['long_answer']['start_token'] >= 0

        long_answers = [
            a['long_answer']
            for a in item['annotations']
            if a['long_answer']['start_token'] >= 0 and has_long_answer
        ]
        if has_long_answer:
            start_token = long_answers[0]['start_token']
            end_token = long_answers[0]['end_token']
            x = document_text.split(' ')
            long_answer = ' '.join(x[start_token:end_token])
            long_answer = re.sub('<[^<]+?>', '', long_answer).replace('\n', '').strip()
        arr.append(long_answer) if has_long_answer else arr.append('')

        # short_answer
        has_short_answer = annotation['short_answers'] or annotation['yes_no_answer'] != 'NONE'
        short_answers = [
            a['short_answers']
            for a in item['annotations']
            if a['short_answers'] and has_short_answer
        ]
        if has_short_answer and len(annotation['short_answers']) != 0:
            sa = []
            for i in short_answers[0]:
                start_token_s = i['start_token']
                end_token_s = i['end_token']
                shorta = ' '.join(x[start_token_s:end_token_s])
                sa.append(shorta)
            short_answer = '|'.join(sa)
            short_answer = re.sub('<[^<]+?>', '', short_answer).replace('\n', '').strip()
        arr.append(short_answer) if has_short_answer else arr.append('')

        ## title
        arr.append(item['document_title'])

        ## abs
        if document_text.find('<P>') != -1:
            abs_start = document_text.index('<P>')
            abs_end = document_text.index('</P>')
            abs = document_text[abs_start+3:abs_end]
        else:
            abs = ''
        arr.append(abs)

        ## content
        if document_text.rfind('</Ul>') != -1:
            final = document_text.rindex('</Ul>')
            document_text = document_text[:final]
            if document_text.rfind('</Ul>') != -1:
                final = document_text.rindex('</Ul>')
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
            else:
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
        else:
            content = document_text[abs_end+4:]
            content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
            content = re.sub(' +', ' ', content)
            arr.append(content)
        doc_tac = item['document_title'] + abs + content
        arr.append(doc_tac)
        language = 'en'
        arr.append(language)
        nq_dev.append(arr)

nq_dev_df = pd.DataFrame(nq_dev)
nq_dev_df.to_csv(r"nq_dev.tsv", sep="\t", mode = 'w', header=None, index =False)

7830it [02:20, 55.69it/s]


In [4]:
nq_train = []
with gzip.open("/home/ec2-user/quic-efs/user/bowenjin/seq2seq/retrieval-data/NQ/v1.0-simplified_simplified-nq-train.jsonl.gz", "r+") as f:
    for item in jsonlines.Reader(f):
        ## question_text
        arr = []
        question_text = item['question_text']
        arr.append(question_text)

        ## example_id
        example_id = str(item['example_id'])
        arr.append(example_id)
        
        document_text = item['document_text']
        
        ## long_answer
        annotation = item['annotations'][0]
        has_long_answer = annotation['long_answer']['start_token'] >= 0

        long_answers = [
            a['long_answer']
            for a in item['annotations']
            if a['long_answer']['start_token'] >= 0 and has_long_answer
        ]
        if has_long_answer:
            start_token = long_answers[0]['start_token']
            end_token = long_answers[0]['end_token']
            x = document_text.split(' ')
            long_answer = ' '.join(x[start_token:end_token])
            long_answer = re.sub('<[^<]+?>', '', long_answer).replace('\n', '').strip()
        arr.append(long_answer) if has_long_answer else arr.append('')

        # short_answer
        has_short_answer = annotation['short_answers'] or annotation['yes_no_answer'] != 'NONE'
        short_answers = [
            a['short_answers']
            for a in item['annotations']
            if a['short_answers'] and has_short_answer
        ]
        if has_short_answer and len(annotation['short_answers']) != 0:
            sa = []
            for i in short_answers[0]:
                start_token_s = i['start_token']
                end_token_s = i['end_token']
                shorta = ' '.join(x[start_token_s:end_token_s])
                sa.append(shorta)
            short_answer = '|'.join(sa)
            short_answer = re.sub('<[^<]+?>', '', short_answer).replace('\n', '').strip()
        arr.append(short_answer) if has_short_answer else arr.append('')

        ## title
        if document_text.find('<H1>') != -1:
            title_start = document_text.index('<H1>')
            title_end = document_text.index('</H1>')
            title = document_text[title_start+4:title_end]
        else:
            title = ''
        arr.append(title)

        ## abs
        if document_text.find('<P>') != -1:
            abs_start = document_text.index('<P>')
            abs_end = document_text.index('</P>')
            abs = document_text[abs_start+3:abs_end]
        else:
            abs = ''
        arr.append(abs)

        ## content
        if document_text.rfind('</Ul>') != -1:
            final = document_text.rindex('</Ul>')
            document_text = document_text[:final]
            if document_text.rfind('</Ul>') != -1:
                final = document_text.rindex('</Ul>')
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
            else:
                content = document_text[abs_end+4:final]
                content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
                content = re.sub(' +', ' ', content)
                arr.append(content)
        else:
            content = document_text[abs_end+4:]
            content = re.sub('<[^<]+?>', '', content).replace('\n', '').strip()
            content = re.sub(' +', ' ', content)
            arr.append(content)

        doc_tac = title + abs + content
        arr.append(doc_tac)

        language = 'en'
        arr.append(language)
        nq_train.append(arr)

nq_train_df = pd.DataFrame(nq_train)
nq_train_df.to_csv(r"nq_train.tsv", sep="\t", mode = 'w', header=None, index =False)

In [2]:
## Mapping tool

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def lower(x):
    text = tokenizer.tokenize(x)
    id_ = tokenizer.convert_tokens_to_ids(text)
    return tokenizer.decode(id_)

In [3]:
## doc_tac denotes the concatenation of title, abstract and content

nq_dev = pd.read_csv('nq_dev.tsv', \
                     names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                     header=None, sep='\t')

nq_train = pd.read_csv('nq_train.tsv', \
                       names=['query', 'id', 'long_answer', 'short_answer', 'title', 'abstract', 'content', 'doc_tac', 'language'],\
                       header=None, sep='\t')


In [4]:
nq_dev['title'] = nq_dev['title'].map(lower)
nq_train['title'] = nq_train['title'].map(lower)

In [5]:
## Concat train doc and validation doc to obtain full document collection

#nq_all_doc = nq_train.append(nq_dev)
nq_all_doc = pd.concat([nq_train, nq_dev])
nq_all_doc.reset_index(inplace = True)

print(len(nq_train))
print(len(nq_dev))

307373
7830


In [6]:
## Remove duplicated documents based on titles

nq_all_doc.drop_duplicates('title', inplace = True)
nq_all_doc.reset_index(inplace = True)

In [9]:
## The total amount of documents : 109739

len(nq_all_doc)

109739

In [55]:
## Construct mapping relation

title_doc = {}
title_doc_id = {}
id_doc = {}
id_doc_all = {}
ran_id_old_id = {}
idx = 0
doc_id = {}
title_id = {}

for i in range(len(nq_all_doc)):
    title_doc[nq_all_doc['title'][i]] =  nq_all_doc['doc_tac'][i]
    title_doc_id[nq_all_doc['title'][i]] = idx
    id_doc[idx] = nq_all_doc['doc_tac'][i]
    id_doc_all[idx] = {'id': idx, 'title': nq_all_doc['title'][i], 'abstract': nq_all_doc['abstract'][i], 'content': nq_all_doc['content'][i]}
    doc_id[nq_all_doc['doc_tac'][i]] = idx

    assert nq_all_doc['title'][i] not in title_id
    title_id[nq_all_doc['title'][i]] = idx

    ran_id_old_id[idx] = nq_all_doc['id'][i]
    idx += 1

In [11]:
## Construct Document Content File

train_file = open("corpus.tsv", 'w') 

for docid in id_doc.keys():
    train_file.write('\t'.join([str(docid), id_doc[docid]]) + '\n')
    train_file.flush()

In [ ]:
## Construct Document Content File

with open("corpus.json", 'w') as train_file:
    for docid in id_doc_all.keys():
        #train_file.write('\t'.join([str(docid), id_doc[docid]]) + '\n')
        train_file.write(json.dumps(id_doc_all[docid]) + '\n')

In [46]:
## Transfer original train/val/test files into good format

doc2id = {}
id2doc = {}

with open("corpus.tsv") as f:
    readin = f.readlines()
    for line in tqdm(readin):
        tmp = line.strip().split('\t')
        assert len(tmp) == 2
        assert tmp[1] not in doc2id
        doc2id[tmp[1]] = tmp[0]
        id2doc[tmp[0]] = tmp[1]

100%|██████████| 109739/109739 [00:05<00:00, 18353.43it/s]


In [61]:
## process train

train_dict = defaultdict(set)
for iid in tqdm(range(len(nq_train))):
    train_dict[nq_train['query'][iid]].add(title_id[nq_train['title'][iid]])

stats = 0
with open('train.csv', 'w') as fout:
    for query in train_dict:
        if len(train_dict[query]) > 1:
            stats += 1
        fout.write(query + '\t' + ','.join([str(ii) for ii in list(train_dict[query])]) + '\n')

print(f'query with more than 1 ground truth doc: {stats}/{len(train_dict)}')


100%|██████████| 307373/307373 [00:03<00:00, 101965.10it/s]


query with more than 1 ground truth doc: 0/307373


In [63]:
## process test

dev_dict = defaultdict(set)
for iid in tqdm(range(len(nq_dev))):
    dev_dict[nq_dev['query'][iid]].add(title_id[nq_dev['title'][iid]])

stats = 0
with open('test.csv', 'w') as fout:
    for query in dev_dict:
        if len(dev_dict[query]) > 1:
            stats += 1
        fout.write(query + '\t' + ','.join([str(ii) for ii in list(dev_dict[query])]) + '\n')

print(f'query with more than 1 ground truth doc: {stats}/{len(dev_dict)}')

100%|██████████| 7830/7830 [00:00<00:00, 109717.10it/s]

query with more than 1 ground truth doc: 0/7830
